**Regras de associação**

**Algoritmo Apriori**

**Veja: https://www.youtube.com/watch?v=0v6PiOpsJi8**

In [131]:
#!pip install apyori

In [132]:
import pandas as pd
from apyori import apriori

In [133]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('./PãoeManteiga SimNao.csv', sep=';',encoding='cp1252', header=0)
base

,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
0,Não,Sim,Não,Sim,Sim,Não,Não
1,Sim,Não,Sim,Sim,Sim,Não,Não
2,Não,Sim,Não,Sim,Sim,Não,Não
3,Sim,Sim,Não,Sim,Sim,Não,Não
4,Não,Não,Sim,Não,Não,Não,Não
5,Não,Não,Não,Não,Sim,Não,Não
6,Não,Não,Não,Sim,Não,Não,Não
7,Não,Não,Não,Não,Não,Não,Sim
8,Não,Não,Não,Não,Não,Sim,Sim
9,Não,Não,Não,Não,Não,Sim,Não


In [134]:
base.shape

(10, 7)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [135]:
# 2. Gerar a Lista de Transações (CORREÇÃO CRUCIAL)
transacoes = []

# Iterar sobre as linhas do DataFrame
for index, row in base.iterrows():
    compra = []
    for col in base.columns:
        if row[col] == 'Sim':
            compra.append(col)

    if compra:
        transacoes.append(compra)

print("Exemplo das 5 primeiras transações no formato correto:")
print(transacoes[:5])

Exemplo das 5 primeiras transações no formato correto:
[['Café', 'Pão', 'Manteiga'], ['Leite', 'Cerveja', 'Pão', 'Manteiga'], ['Café', 'Pão', 'Manteiga'], ['Leite', 'Café', 'Pão', 'Manteiga'], ['Cerveja']]


In [136]:
transacoes

[['Café', 'Pão', 'Manteiga'],
 ['Leite', 'Cerveja', 'Pão', 'Manteiga'],
 ['Café', 'Pão', 'Manteiga'],
 ['Leite', 'Café', 'Pão', 'Manteiga'],
 ['Cerveja'],
 ['Manteiga'],
 ['Pão'],
 ['Feijão'],
 ['Arroz', 'Feijão'],
 ['Arroz']]

In [137]:
type(transacoes)

list

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [138]:
regras = apriori(transacoes, min_support = 0.3, min_confidence = 0.6, min_length = 1)
saida = list(regras)
print(len(saida))
print(saida)

4
[RelationRecord(items=frozenset({'Manteiga', 'Café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0), OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Café'}), confidence=0.6, lift=2.0)]), RelationRecord(items=frozenset({'Pão', 'Café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Pão'}), confidence=1.0, lift=2.0), OrderedStatistic(items_base=frozenset({'Pão'}), items_add=frozenset({'Café'}), confidence=0.6, lift=2.0)]), RelationRecord(items=frozenset({'Pão', 'Manteiga'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Pão'}), confidence=0.8, lift=1.6), OrderedStatistic(items_base=frozenset({'Pão'}), items_add=frozenset({'Manteiga'}), confidence=0.8, lift=1.6)]), RelationRecord(items=frozenset({'Pão', 'Manteiga', 'Café'}), support=0.3, ordered_

In [139]:
#Mostrar itemsets por tamanho
from collections import Counter
itemsets_por_tamanho = {}
min_support_percentual = 0.3
total_transacoes = len(transacoes)
min_support_count = min_support_percentual * total_transacoes

todos_os_itens = [item for sublist in transacoes for item in sublist]

contagem_itens = Counter(todos_os_itens)

itemsets_tamanho_1 = []
for item, contagem in contagem_itens.items():
    if contagem >= min_support_count:
        itemsets_tamanho_1.append({item})

print("\n--- Itemsets de Tamanho 1 ---")
print(f"Suporte Mínimo (Contagem): {min_support_count:.2f} (≈ {min_support_percentual*100}%)")
for itemset in itemsets_tamanho_1:
    item = list(itemset)[0]
    suporte = contagem_itens[item] / total_transacoes
    print(f"[{item}] - Suporte: {suporte:.3f} (Contagem: {contagem_itens[item]})")

for record in saida:
    itemset = set(record.items)
    tamanho_k = len(itemset)
    if tamanho_k not in itemsets_por_tamanho:
        itemsets_por_tamanho[tamanho_k] = []
        
    itemsets_por_tamanho[tamanho_k].append(itemset)

for k in sorted(itemsets_por_tamanho.keys()):
    print(f"\n--- Itemsets de Tamanho {k} ({len(itemsets_por_tamanho[k])} Total) ---")
    for itemset in itemsets_por_tamanho[k]:
        print(list(itemset))


--- Itemsets de Tamanho 1 ---
Suporte Mínimo (Contagem): 3.00 (≈ 30.0%)
[Café] - Suporte: 0.300 (Contagem: 3)
[Pão] - Suporte: 0.500 (Contagem: 5)
[Manteiga] - Suporte: 0.500 (Contagem: 5)

--- Itemsets de Tamanho 2 (3 Total) ---
['Manteiga', 'Café']
['Pão', 'Café']
['Pão', 'Manteiga']

--- Itemsets de Tamanho 3 (1 Total) ---
['Pão', 'Manteiga', 'Café']


In [140]:
print(saida[0])


RelationRecord(items=frozenset({'Manteiga', 'Café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0), OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Café'}), confidence=0.6, lift=2.0)])


In [141]:
print(saida[1])

RelationRecord(items=frozenset({'Pão', 'Café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Pão'}), confidence=1.0, lift=2.0), OrderedStatistic(items_base=frozenset({'Pão'}), items_add=frozenset({'Café'}), confidence=0.6, lift=2.0)])


In [142]:
print(saida[2])

RelationRecord(items=frozenset({'Pão', 'Manteiga'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Manteiga'}), items_add=frozenset({'Pão'}), confidence=0.8, lift=1.6), OrderedStatistic(items_base=frozenset({'Pão'}), items_add=frozenset({'Manteiga'}), confidence=0.8, lift=1.6)])


**Para facilitar a visualização, veja como transformar isso em um dataframe**

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [143]:
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []
RegrasFinais = []

for resultado in saida:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [144]:
RegrasFinais

,Antecedente,Consequente,suporte,confianca,lift
0,[Café],[Manteiga],0.3,1.00,2.0
1,[Manteiga],[Café],0.3,0.60,2.0
2,[Café],[Pão],0.3,1.00,2.0
3,[Pão],[Café],0.3,0.60,2.0
4,[Manteiga],[Pão],0.4,0.80,1.6
5,[Pão],[Manteiga],0.4,0.80,1.6
6,[Café],"[Pão, Manteiga]",0.3,1.00,2.5
7,[Manteiga],"[Pão, Café]",0.3,0.60,2.0
8,[Pão],"[Manteiga, Café]",0.3,0.60,2.0
9,"[Manteiga, Café]",[Pão],0.3,1.00,2.0


**Podemos ordenar estas regras por uma métrica desejada**

In [145]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Antecedente,Consequente,suporte,confianca,lift
6,[Café],"[Pão, Manteiga]",0.3,1.00,2.5
11,"[Pão, Manteiga]",[Café],0.3,0.75,2.5
0,[Café],[Manteiga],0.3,1.00,2.0
1,[Manteiga],[Café],0.3,0.60,2.0
3,[Pão],[Café],0.3,0.60,2.0
2,[Café],[Pão],0.3,1.00,2.0
8,[Pão],"[Manteiga, Café]",0.3,0.60,2.0
7,[Manteiga],"[Pão, Café]",0.3,0.60,2.0
9,"[Manteiga, Café]",[Pão],0.3,1.00,2.0
10,"[Pão, Café]",[Manteiga],0.3,1.00,2.0
